In [316]:
import time
import requests
from bs4 import BeautifulSoup
import pymongo
import pandas as pd
import numpy as np
import re

from sklearn.externals import joblib

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
wnlzer = WordNetLemmatizer()


from pprint import pprint
from time import time
import logging


from collections import defaultdict
from gensim import corpora, models
from gensim.similarities import MatrixSimilarity

import sys
import os
import csv
import shutil
from collections import defaultdict

from sklearn.linear_model import SGDClassifier, Perceptron
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV, ParameterGrid
from sklearn.pipeline import Pipeline
from sklearn.datasets import load_files
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import roc_auc_score
#import parfit.parfit as pf

import matplotlib.pyplot as plt

In [144]:
#Import the dataframe with strain review info

test_reviews = pd.read_csv("./../../new_full_test_strains_reviews.csv")

#Drop Spam users

new_tr = test_reviews.drop([3570,2728,355], axis=0)

new_tr.columns

In [ ]:
#Look for spam strings in the df[columns](i.e. in a review sring)...return user or strain etc that has it. 

mask = np.column_stack([new_tr[col].str.contains(r"ke thick robust problem going coughing thats alway", na=False) for col in new_tr])
new_tr.loc[mask.any(axis=1)]

In [370]:
#Drop dupplicate colums and user index

to_drop = ['Unnamed: 0','hybrid/blue-dream.1',
       'sativa/sour-diesel.1', 'hybrid/gsc.1', 'sativa/green-crack.1',
       'hybrid/og-kush.1', 'indica/granddaddy-purple.1',
       'hybrid/original-glue.1', 'hybrid/white-widow.1','hybrid/skywalker-og']

tr = new_tr.drop(to_drop, axis=1) 

#Save Strains as index for later...

final_strain_index = tr.columns
final_strain_index

Index(['hybrid/blue-dream', 'sativa/sour-diesel', 'hybrid/gsc',
       'sativa/green-crack', 'hybrid/og-kush', 'indica/granddaddy-purple',
       'hybrid/original-glue', 'hybrid/white-widow', 'sativa/jack-herer',
       'indica/bubba-kush', 'hybrid/pineapple-express', 'hybrid/trainwreck',
       'hybrid/ak-47', 'sativa/durban-poison', 'indica/northern-lights',
       'hybrid/headband', 'indica/blue-cheese', 'sativa/strawberry-cough',
       'hybrid/chemdawg', 'indica/purple-kush', 'sativa/lemon-haze',
       'sativa/super-lemon-haze', 'indica/grape-ape', 'indica/blueberry',
       'sativa/alaskan-thunder-fuck', 'sativa/super-silver-haze',
       'indica/blackberry-kush', 'hybrid/cherry-pie', 'indica/master-kush',
       'hybrid/cheese', 'indica/death-star', 'sativa/chocolope',
       'sativa/amnesia-haze', 'hybrid/tahoe-og', 'sativa/maui-wowie',
       'hybrid/platinum-gsc', 'sativa/harlequin', 'indica/gods-gift',
       'indica/la-confidential', 'hybrid/agent-orange', 'indica/purple-u

In [376]:
strain_lookup = {}
for strain, cnt in zip(list(final_strain_index),range(len(list(final_strain_index)))):
    strain_lookup[cnt] = strain

In [339]:
# Convert list of strain reviews strings into a single string for each strain;
# store strain string in list of all strain strings.

def cols_2_string(df):
    strain_list = []
    cols = df.columns
    for i in cols:
        strain = df[i].dropna().tolist()
        strain_str = " ".join(strain)
        strain_list.append(strain_str)
    return strain_list

col_string = cols_2_string(tr)
col_string

In [340]:
#Split each review it by white space, for each strain

texts = [[word for word in document.split()]for document in col_string]

#Count word frequencies for each strain

from collections import defaultdict
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

#frequency

In [342]:
# Only keep words that appear more than once
processed_corpus = [[token for token in text if frequency[token] > 1] for text in texts]

print(processed_corpus)

In [305]:
#Create dictionary from corpus words

dictionary = corpora.Dictionary(processed_corpus)
#pprint(dictionary)


# store the dictionary, for future reference

dictionary.save('new_strains.dict')  

2019-04-10 16:04:54,137 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2019-04-10 16:04:54,290 : INFO : built Dictionary(5822 unique tokens: ['ab', 'able', 'absolute', 'absolutely', 'abundance']...) from 47 documents (total 149613 corpus positions)
2019-04-10 16:04:54,291 : INFO : saving Dictionary object under new_strains.dict, separately None
2019-04-10 16:04:54,293 : WARNING : this function is deprecated, use smart_open.open instead
2019-04-10 16:04:54,298 : INFO : saved new_strains.dict


In [343]:
#Print the distinct words in the corus and their corresponding id

print(dictionary.token2id)

{'ab': 0, 'able': 1, 'absolute': 2, 'absolutely': 3, 'abundance': 4, 'accompanied': 5, 'accomplish': 6, 'accurate': 7, 'ache': 8, 'acheter': 9, 'achieve': 10, 'across': 11, 'acting': 12, 'active': 13, 'activity': 14, 'actual': 15, 'actually': 16, 'acute': 17, 'add': 18, 'adderall': 19, 'adhd': 20, 'adjust': 21, 'adventurous': 22, 'advertised': 23, 'advice': 24, 'af': 25, 'afternoon': 26, 'afterwards': 27, 'age': 28, 'ago': 29, 'ah': 30, 'ahead': 31, 'ailment': 32, 'aint': 33, 'airy': 34, 'aisle': 35, 'aka': 36, 'ale': 37, 'alert': 38, 'alike': 39, 'alive': 40, 'allday': 41, 'allowed': 42, 'allowing': 43, 'allows': 44, 'alltime': 45, 'almost': 46, 'alone': 47, 'along': 48, 'alotta': 49, 'already': 50, 'also': 51, 'although': 52, 'always': 53, 'amazing': 54, 'amber': 55, 'amongst': 56, 'amount': 57, 'amplify': 58, 'anger': 59, 'angry': 60, 'animal': 61, 'ankle': 62, 'another': 63, 'anti': 64, 'antianxiety': 65, 'anticipating': 66, 'anxiety': 67, 'anxious': 68, 'anymore': 69, 'anyone': 70

In [369]:
# dob2bow counts the number of occurrences of each distinct word, 
# converts the word to its integer word id and returns the result as a sparse vector
# returns the sparse vector for each strain in a list indexed by strains

corpus = [dictionary.doc2bow(text) for text in texts]
corpus

len(corpus)

In [400]:
#Initiate TF-IDF model
#calculate tf-idf weights and matrix for strain lis 
tfidf = models.TfidfModel(corpus)

2019-04-10 21:27:47,891 : INFO : collecting document frequencies
2019-04-10 21:27:47,893 : INFO : PROGRESS: processing document #0
2019-04-10 21:27:47,916 : INFO : calculating IDF weights for 47 documents and 5821 features (51544 matrix non-zeros)


In [399]:
#Transform the corpus list, fit it 

corpus_tfidf = tfidf[corpus]
corpus_tfidf

for doc in corpus_tfidf:
    print(doc)

In [402]:
#Initiate the LSI (Latent Semnatic Indexing) model with dictionary that i create earlier and transsformed fit corpus

model = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=300)

model

2019-04-10 21:34:25,013 : INFO : using serial LSI version on this node
2019-04-10 21:34:25,020 : INFO : updating model with new documents
2019-04-10 21:34:25,299 : INFO : preparing a new chunk of documents
2019-04-10 21:34:25,314 : INFO : using 100 extra samples and 2 power iterations
2019-04-10 21:34:25,316 : INFO : 1st phase: constructing (5822, 400) action matrix
2019-04-10 21:34:25,366 : INFO : orthonormalizing (5822, 400) action matrix
2019-04-10 21:34:26,099 : INFO : 2nd phase: running dense svd on (400, 47) matrix
2019-04-10 21:34:26,111 : INFO : computing the final decomposition
2019-04-10 21:34:26,112 : INFO : keeping 47 factors (discarding 0.000% of energy spectrum)
2019-04-10 21:34:26,130 : INFO : processed documents up to #47
2019-04-10 21:34:26,131 : INFO : topic #0(1.934): 0.160*"lemon" + 0.123*"cheese" + 0.120*"haze" + 0.118*"diesel" + 0.104*"blue" + 0.101*"kush" + 0.100*"blueberry" + 0.099*"og" + 0.097*"gsc" + 0.096*"sour"
2019-04-10 21:34:26,133 : INFO : topic #1(1.270

In [401]:
#User input part...
user_input = "makes my farts stink"

#Create BOW and LSI vector for the user input to

vec_bow = dictionary.doc2bow(user_input.lower().split())
vec_lsi = model[vec_bow]
#print(vec_lsi)

In [407]:
# Create (nxn) strain similarity matrix to index from corpus
# transform corpus to LSI space 


#index = MatrixSimilarity(model[corpus]) 
#index
#index.save('strain_sim.index')
index = MatrixSimilarity.load('strain_sim.index')

2019-04-10 21:50:17,993 : INFO : loading MatrixSimilarity object from strain_sim.index
2019-04-10 21:50:17,996 : WARNING : this function is deprecated, use smart_open.open instead
2019-04-10 21:50:17,999 : INFO : loaded strain_sim.index


In [403]:
#Index into the similarity matrix with the input vector
sims = index[vec_lsi]

#print(list(enumerate(sims)))

#sort the similarity list
sims_sort = sorted(enumerate(sims), key=lambda item: -item[1])

#return the top 5 strains and their score
top_5 = sims_sort[:5]
for i in top_5:
    print(strain_lookup[i[0]],i[1])

sativa/sour-diesel 0.70947397
indica/blueberry 0.35766658
hybrid/chemdawg 0.34080726
hybrid/golden-goat 0.33501703
hybrid/ak-47 0.25149664
